In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import random
import json
import os

In [ ]:
# %cd ..

In [ ]:
config_path = 'data/Heino_test/output_20240829_164635/voxel_resolution15_area9/config.json'

In [ ]:
with open(config_path, 'r') as file:
    CONFIG = json.load(file)
    print(CONFIG)

In [ ]:
folder_path = CONFIG['study_area']['data_root']
data_root = os.path.join(folder_path, CONFIG['output_folder_name'])
point_grid_path = os.path.join(data_root, 'intermediate', 'point_grid.dat')
index_map_path = os.path.join(data_root,  'index_map.dat')
shadow_map_path = os.path.join(data_root, 'shadow_map.dat')


horizon_factor_path = os.path.join(data_root, 'horizon_factor_map.dat')
sky_view_factor_path = os.path.join(data_root,  'sky_view_factor_map.dat')
cosine_map_path = os.path.join(data_root, 'cosine_map.dat')

voxel_size = CONFIG['voxel_resolution']
num_azimuth = 360//CONFIG['azimuth_resolution']
num_elevation = 90//CONFIG['elevation_resolution']
print(num_azimuth, num_elevation)
num_samples = num_azimuth*num_elevation
index_dim = (num_elevation,num_azimuth)

estimated_irradiance_path = os.path.join(data_root, 'irradiance.npy')
weather_data_path = os.path.join(data_root, 'weather_data.npy')

point_grid = np.loadtxt(point_grid_path)
estimated_irradiance = np.load(estimated_irradiance_path)
weather_data = np.load(weather_data_path)

In [ ]:
def print_point_info(point_grid, horizon_factor_path, sky_view_factor_path, index_map_path, cosine_map_path, index_dim, point_idx):
    horizon_factor_path = np.memmap(horizon_factor_path, dtype=np.float16, mode='r')
    sky_view_factor_path = np.memmap(sky_view_factor_path, dtype=np.int32, mode='r')
    index_map = np.memmap(index_map_path, dtype=np.uint32, mode='r')
    # cosine_map = np.memmap(cosine_map_path, dtype=np.float16, mode='r')

    num_samples = index_dim[0]*index_dim[1]
    index_start = point_idx*num_samples
    index_end = (point_idx+1)*num_samples

    voxel_indices = index_map[index_start:index_end]
    # breakpoint()
    voxel_indices = voxel_indices.reshape(index_dim)
    unique_values = np.unique(voxel_indices)

    # sum_of_cosine = np.sum(cosine_map[index_start*6:index_end*6])
    # print("sum of cosine: ", sum_of_cosine)

    # 创建一个随机颜色映射，确保白色为最大值对应的颜色
    colors = {}
    for value in unique_values:
        # print(value)
        if value == 4294967295:
            colors[value] = (1.0, 1.0, 1.0)  # 白色
        else:
            colors[value] = (random.random(), random.random(), random.random())

    # 将图像数据转换为 RGB
    image = np.zeros((voxel_indices.shape[0], voxel_indices.shape[1], 3))

    for i in range(voxel_indices.shape[0]):
        for j in range(voxel_indices.shape[1]):
            image[i, j] = colors[voxel_indices[i, j]]

    image = np.flipud(image)
    plt.xticks(np.linspace(0, voxel_indices.shape[1] - 1, num=5), 
           labels=np.linspace(0, 360, num=5).astype(int))  # 横轴 azimuth 从 0 到 360 度

    plt.yticks(np.linspace(0, voxel_indices.shape[0] - 1, num=4), 
            labels=np.linspace(90, 0, num=4).astype(int))  # 纵轴 elevation 从 0 到 90 度

    # 添加坐标轴标签
    plt.xlabel('Azimuth (degrees)')
    plt.ylabel('Elevation (degrees)')

    plt.imshow(image)
    # plt.savefig('BK.png', bbox_inches='tight', pad_inches=0, dpi=300)
    # plt.axis('off')  # 隐藏坐标轴
    plt.show()

    print("sky view factor: ", sky_view_factor_path[point_idx]/num_samples)
    print("horizon factor: ", horizon_factor_path[point_idx])
    formatted_values = [f"{x:.5f}" for x in point_grid[point_idx]]
    print("point info: ", formatted_values)

    



In [ ]:
idx = 

In [ ]:
print_point_info(point_grid, horizon_factor_path, sky_view_factor_path, index_map_path, cosine_map_path, index_dim, idx)

In [ ]:
plt.figure(figsize=(12, 8))

for i in range(6):
    plt.plot(estimated_irradiance[i][idx][0:8], label=f"Bounce {i}", marker='o')


plt.plot(weather_data[:,0][0:8], label="Weather Data [:,0]", marker='o')

plt.xlabel("Index")
plt.ylabel("Value")
plt.title("Comparison of Estimated Irradiance and Weather Data")
plt.legend()
plt.grid(True)
plt.savefig('Heino_9_5_15.png', bbox_inches='tight', pad_inches=0, dpi=300)
plt.show()